## Topographic Complexity/Variability: Terrain Ruggedness Index (TRI)  
Developed in November 2023 by Dr. Larry Syu-Heng Lai (University of Washington)  

Recommended reference:
* Wilson, M.F.J., O’Connell, B., Brown, C., Guinan, J.C., Grehan, A.J., 2007. Multiscale Terrain Analysis of Multibeam Bathymetry Data for Habitat Mapping on the Continental Slope. Marine Geodesy 30, 3-35. https://doi.org/10.1080/01490410701295962 

### Initial setup

In [34]:
import numpy as np
from scipy.ndimage import generic_filter
from numba import jit
from tqdm.notebook import tqdm
import itertools
import rasterio

### Define data path

In [35]:
# Define your file paths and file names separately
input_folder = '/Users/larryslai/Library/CloudStorage/Dropbox/QGIS/WA LiDAR/'
input_file_name = 'Test_DEM.tif'
#input_file_name = 'Tokeland_DEM.tif'
#input_file_name = 'Nemah_DEM.tif'
#input_file_name = 'Francies_DEM.tif'

output_folder = '/Users/larryslai/Library/CloudStorage/Dropbox/QGIS/WA LiDAR/'
output_file_name = 'Test_pyTRI.tif'
#output_file_name = 'Tokeland_pyTRI.tif'
#output_file_name = 'Nemah_pyTRI.tif'
#output_file_name = 'Francies_pyTRI.tif'

# Combine folder and file names to create the full paths
input_tif_path = input_folder + input_file_name
output_tif_path = output_folder + output_file_name

### Read a DEM

In [36]:
with rasterio.open(input_tif_path) as src:
    dem = src.read(1)  # Read the first band into a 2D array
    meta = src.meta

See coordinate system info of the GeoTIFF

In [37]:
# Open the GeoTIFF file
with rasterio.open(input_tif_path) as src:
    # Read the CRS
    crs = src.crs
    
    # Print the CRS information
    print(f"CRS: {crs}")
    print(f"CRS as WKT: {crs.wkt}")
    print(f"CRS as PROJ string: {crs.to_proj4()}")
    print(f"CRS as EPSG code: {crs.to_epsg()}")
    print(f"CRS as dictionary: {crs.to_dict()}")

CRS: PROJCS["NAD83(HARN) / Washington South (ftUS)",GEOGCS["NAD83(HARN)",DATUM["North_American_1983_HARN",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",45.3333333333333],PARAMETER["central_meridian",-120.5],PARAMETER["standard_parallel_1",45.8333333333333],PARAMETER["standard_parallel_2",47.3333333333333],PARAMETER["false_easting",1640416.66666667],PARAMETER["false_northing",0],UNIT["US survey foot",0.304800609601219,AUTHORITY["EPSG","9003"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
CRS as WKT: PROJCS["NAD83(HARN) / Washington South (ftUS)",GEOGCS["NAD83(HARN)",DATUM["North_American_1983_HARN",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitud

## Terrain Ruggedness Index (TRI)

The Terrain Ruggedness Index (TRI) is a measure used to quantify the topographic heterogeneity or complexity of a landscape. It is calculated by summarizing the change in elevation between a central pixel and its surrounding pixels within a specified window. The TRI is defined for a window of size $ n \times n $ by the following equation:

$$
TRI(n) = \frac{\sum_{i=-N}^{N} \sum_{j=-N}^{N} | Z_{ij} - Z_{00} |}{n^2 - 1}
$$

where:
- $Z_{ij}$ represents the elevation of the pixel at the $i^{th}$ row and $j^{th}$ column within the window,
- $Z_{00}$ is the elevation of the central pixel,
- $N$ is defined as $(n-1) / 2$, representing the extent of the window around the central pixel.

TRI provides a single value representing the average absolute difference in elevation between the central pixel and its neighbors, thereby characterizing the local roughness or ruggedness of the terrain.


##### Terrain Ruggedness Index (TRI) function  
* Enabling Numba to speed up the execution of functions that use NumPy arrays and loops

In [38]:
# Shared counter that will be updated during the TRI calculation
counter = itertools.count()

# Define a Numba-optimized function for TRI calculation
@jit(nopython=True)
def tri_func(window, window_size):
    center_pixel = window[window_size**2 // 2]
    diff_sum = 0.0
    for w in window:
        diff_sum += np.abs(w - center_pixel)
    return diff_sum / (window_size**2 - 1)

def tri_filter_wrapper(dem, window_size):
    """
    Wrapper for the generic_filter function to update the progress bar.
    
    :param dem: A 2D numpy array of the DEM.
    :param window_size: The size of the window to calculate TRI (must be an odd number).
    :return: A 2D numpy array of the TRI.
    """
    def filter_func(window):
        next(counter)  # Update the shared counter
        return tri_func(window, window_size)
    
    return generic_filter(dem, filter_func, size=(window_size, window_size))

def calculate_TRI(dem, window_size):
    """
    Calculate the Terrain Ruggedness Index (TRI) for a digital elevation model (DEM).
    
    :param dem: A 2D numpy array of the DEM.
    :param window_size: The size of the window to calculate TRI (must be an odd number).
    :return: A 2D numpy array of the TRI.
    """
    # Reset the shared counter
    global counter
    counter = itertools.count()
    
    # Calculate the total number of windows to process
    total_windows = (dem.shape[0] - window_size + 1) * (dem.shape[1] - window_size + 1)
    
    # Create a tqdm progress bar
    pbar = tqdm(total=total_windows, desc='Calculating TRI')
    
    # Apply the TRI filter wrapper and update the progress bar
    TRI = tri_filter_wrapper(dem, window_size)
    
    # Update the progress bar
    pbar.update(total_windows)
    pbar.close()
    
    return TRI

Calculate TRI with a given window

In [39]:
# Calculate with a N x N window as an example
window_size = 3 # Replace with the desired window size
TRI = calculate_TRI(dem, window_size)

Calculating TRI:   0%|          | 0/1849766 [00:00<?, ?it/s]

Output data into GeoTIFF
* Enabling geotiff compression to reduce writing time
* Enabling Tile-based writing if needed
* Enabling BIGTIFF parameter to allow writing a large GeoTIFF

In [40]:
# Update metadata for output GeoTIFF
meta.update(dtype=rasterio.float32, compress='lzw', tiled=True, bigtiff='IF_SAFER')

# Write TRI to a new GeoTIFF
with rasterio.open(output_tif_path, 'w', **meta) as dst:
    dst.write(TRI.astype(rasterio.float32), 1)